# Modelo SIR (Epidemiológico)

O modelo SIR divide a população em três compartimentos:
- **S**: Suscetíveis
- **I**: Infectados
- **R**: Recuperados

## Equações Diferenciais

$$\frac{dS}{dt} = -\beta \frac{SI}{N}$$

$$\frac{dI}{dt} = \beta \frac{SI}{N} - \gamma I$$

$$\frac{dR}{dt} = \gamma I$$

Onde:
- $N = S + I + R$ (população total, constante)
- $\beta$: taxa de transmissão
- $\gamma$: taxa de recuperação
- $R_0 = \beta/\gamma$: número básico de reprodução

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, FloatSlider, IntSlider
%matplotlib inline

In [2]:
def sir(u, t, beta, gamma, N):
    """Modelo SIR"""
    S, I, R = u
    dS = -beta * S * I / N
    dI = beta * S * I / N - gamma * I
    dR = gamma * I
    return [dS, dI, dR]

## Simulação Interativa

Mexa nos sliders para ver como a epidemia muda!

In [3]:
@interact(
    beta=FloatSlider(min=0.1, max=1.0, step=0.05, value=0.4, description='β (transm.)'),
    gamma=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.1, description='γ (recup.)'),
    N=IntSlider(min=1000, max=50000, step=1000, value=10000, description='N (pop.)'),
    I0=IntSlider(min=1, max=500, step=10, value=10, description='I₀ (infect.)'),
    t_final=IntSlider(min=50, max=500, step=25, value=200, description='Tempo (dias)')
)
def plot_sir(beta, gamma, N, I0, t_final):
    # Simulação com scipy
    S0 = N - I0
    R0_calc = beta / gamma
    t = np.linspace(0, t_final, 1000)
    u0 = [S0, I0, 0]
    sol = odeint(sir, u0, t, args=(beta, gamma, N))
    
    S = sol[:, 0]
    I = sol[:, 1]
    R = sol[:, 2]
    
    # Estatísticas
    pico_I = np.max(I)
    t_pico = t[np.argmax(I)]
    total_infectado = R[-1]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Evolução temporal
    ax1.plot(t, S, 'b-', lw=2, label='S (Suscetíveis)')
    ax1.plot(t, I, 'r-', lw=2, label='I (Infectados)')
    ax1.plot(t, R, 'g-', lw=2, label='R (Recuperados)')
    ax1.axvline(t_pico, color='red', ls='--', alpha=0.5, label=f'Pico (dia {t_pico:.0f})')
    ax1.set_xlabel('Tempo (dias)', fontsize=12)
    ax1.set_ylabel('População', fontsize=12)
    status = 'EPIDEMIA!' if R0_calc > 1 else 'Controlada'
    ax1.set_title(f'R₀ = {R0_calc:.2f} → {status}', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Diagrama de fase
    S_limiar = N / R0_calc
    ax2.plot(S, I, 'purple', lw=2, label='Trajetória')
    ax2.plot(S0, I0, 'go', ms=10, label='Início')
    ax2.axvline(S_limiar, color='orange', ls='--', lw=2, label=f'S* = N/R₀ = {S_limiar:.0f}')
    ax2.set_xlabel('Suscetíveis (S)', fontsize=12)
    ax2.set_ylabel('Infectados (I)', fontsize=12)
    ax2.set_title('Diagrama de Fase', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"{'='*50}")
    print(f"R₀ = {R0_calc:.2f}")
    print(f"Pico: {pico_I:.0f} infectados no dia {t_pico:.0f}")
    print(f"Total infectado: {total_infectado:.0f} ({100*total_infectado/N:.1f}%)")
    if R0_calc > 1:
        print(f"Imunidade de rebanho: {100*(1-1/R0_calc):.1f}%")
    print(f"{'='*50}")

interactive(children=(FloatSlider(value=0.4, description='β (transm.)', max=1.0, min=0.1, step=0.05), FloatSli…

## Número Básico de Reprodução

$$R_0 = \frac{\beta}{\gamma}$$

- $R_0 > 1$: cada infectado gera mais de 1 novo caso → epidemia cresce
- $R_0 < 1$: cada infectado gera menos de 1 novo caso → epidemia morre
- $R_0 = 1$: estado endêmico

### Exemplos reais:

| Doença | R₀ | Imunidade de rebanho |
|--------|-----|---------------------|
| Gripe | 1.3 | 23% |
| COVID-19 | 2.5 | 60% |
| Sarampo | 15 | 93% |

## Comparação de R₀

In [4]:
@interact(
    gamma=FloatSlider(min=0.05, max=0.2, step=0.01, value=0.1, description='γ'),
    N=IntSlider(min=1000, max=50000, step=1000, value=10000, description='N'),
    I0=IntSlider(min=1, max=100, step=5, value=10, description='I₀')
)
def plot_r0_comparison(gamma, N, I0):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    S0 = N - I0
    u0 = [S0, I0, 0]
    t = np.linspace(0, 300, 1000)
    
    r0_values = [(0.8, 'gray'), (1.5, 'blue'), (2.5, 'orange'), (4.0, 'red')]
    
    for r0_val, color in r0_values:
        beta_test = r0_val * gamma
        sol = odeint(sir, u0, t, args=(beta_test, gamma, N))
        ax.plot(t, sol[:, 1], color=color, lw=2, label=f'R₀ = {r0_val}')
    
    ax.set_xlabel('Tempo (dias)', fontsize=12)
    ax.set_ylabel('Infectados', fontsize=12)
    ax.set_title('Efeito do R₀ na Curva de Infectados', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.1, description='γ', max=0.2, min=0.05, step=0.01), IntSlider(value=1…

## Imunidade de Rebanho

A epidemia diminui quando $\frac{dI}{dt} < 0$:

$$\beta \frac{SI}{N} < \gamma I \implies S < \frac{N}{R_0}$$

Fração mínima de imunes para controlar:

$$p_c = 1 - \frac{1}{R_0}$$

## Variando Infectados Iniciais

In [ ]:
@interact(
    beta=FloatSlider(min=0.1, max=0.8, step=0.05, value=0.4, description='β'),
    gamma=FloatSlider(min=0.05, max=0.2, step=0.01, value=0.1, description='γ'),
    N=IntSlider(min=5000, max=50000, step=5000, value=10000, description='N')
)
def plot_i0_effect(beta, gamma, N):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    t = np.linspace(0, 300, 1000)
    i0_values = [(1, 'blue'), (10, 'green'), (100, 'orange'), (500, 'red')]
    
    for I0, color in i0_values:
        S0 = N - I0
        u0 = [S0, I0, 0]
        sol = odeint(sir, u0, t, args=(beta, gamma, N))
        ax.plot(t, sol[:, 1], color=color, lw=2, label=f'I₀ = {I0}')
    
    ax.set_xlabel('Tempo (dias)', fontsize=12)
    ax.set_ylabel('Infectados', fontsize=12)
    ax.set_title(f'Efeito do número inicial de infectados (R₀ = {beta/gamma:.2f})', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.4, description='β', max=0.8, min=0.1, step=0.05), FloatSlider(value=…

: 

## Analogia Física

O modelo SIR é análogo a:

1. **Cinética química**: S + I → 2I (infecção), I → R (recuperação)

2. **Conservação**: $S + I + R = N$ constante (como conservação de massa)

3. **Sistema dissipativo**: diferente de Lotka-Volterra, o SIR converge para um estado final estável